In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical



2024-03-26 10:04:57.987937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 10:04:57.988104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 10:04:58.178592: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load data
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')



In [3]:
# Extract labels and images from train data
train_labels = train_data['label']
train_images = train_data.drop('label', axis=1)



In [4]:
# Normalize pixel values to range [0, 1]
train_images = train_images.values.reshape(-1, 28, 28, 1).astype('float32') / 255.0



In [5]:

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels)



In [6]:
# Split train data into train and validation sets
from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)





In [7]:

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
#model.add(layers.Dense(64, activation='relu'))
#model.add(layers.Dropout(0.30))
#model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dropout(0.45))
model.add(layers.Dense(10, activation='softmax'))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [8]:
# Create an instance of the Adam optimizer with a custom learning rate
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [9]:
# Compile the model with the custom optimizer
model.compile(optimizer=custom_optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=25, batch_size=128, validation_split=0.2)



Epoch 1/25
210/210 ━━━━━━━━━━━━━━━━━━━━ 16s 70ms/step - accuracy: 0.4130 - loss: 1.9214 - val_accuracy: 0.8705 - val_loss: 0.5464
Epoch 2/25
210/210 ━━━━━━━━━━━━━━━━━━━━ 19s 64ms/step - accuracy: 0.8290 - loss: 0.5816 - val_accuracy: 0.9243 - val_loss: 0.2859
Epoch 3/25
210/210 ━━━━━━━━━━━━━━━━━━━━ 21s 67ms/step - accuracy: 0.8952 - loss: 0.3586 - val_accuracy: 0.9426 - val_loss: 0.2156
Epoch 4/25
210/210 ━━━━━━━━━━━━━━━━━━━━ 22s 73ms/step - accuracy: 0.9113 - loss: 0.2837 - val_accuracy: 0.9539 - val_loss: 0.1689
Epoch 5/25
210/210 ━━━━━━━━━━━━━━━━━━━━ 19s 68ms/step - accuracy: 0.9308 - loss: 0.2291 - val_accuracy: 0.9597 - val_loss: 0.1436
Epoch 6/25
210/210 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step - accuracy: 0.9423 - loss: 0.1956 - val_accuracy: 0.9628 - val_loss: 0.1257
Epoch 7/25
210/210 ━━━━━━━━━━━━━━━━━━━━ 13s 64ms/step - accuracy: 0.9483 - loss: 0.1761 - val_accuracy: 0.9683 - val_loss: 0.1139
Epoch 8/25
210/210 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.9518 - loss: 0.1548 - 

In [13]:
# Evaluate model on test data
test_images = test_data.values.reshape(-1, 28, 28, 1).astype('float32') / 255.0
predictions = model.predict(test_images)

submission = pd.DataFrame({'ImageId': range(1, len(predictions) + 1), 'Label': np.argmax(predictions, axis=1)})
submission

875/875 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
